In [31]:
# Version 4.13 — Adds MDAnalysis-built index.ndx per run (membrane, protein, protein_membrane, solvent, protein_bb)

import os
import shutil
from contextlib import contextmanager
import COBY
import MDAnalysis as mda
import numpy as np

base_name = "initial_state"
base_dir = "simulations"

protein_pdb   = "6AL2_CG.pdb"      # stays at project root
protein_itp   = "molecule_0.itp"   # copied into each run dir
molecule_type = "molecule_0"

toppar_src = "toppar"

# Relative (run-local) ITP paths that will live inside each run dir
lipid_itp_dir = os.path.join("toppar", "M3-Lipid-Parameters", "ITPs")
m3_core_itp   = os.path.join(lipid_itp_dir, "martini_v3.0.0.itp")
ffbonded_itp  = os.path.join(lipid_itp_dir, "martini_v3.0.0_ffbonded_v2.itp")
pe_itp        = os.path.join(lipid_itp_dir, "martini_v3.0.0_phospholipids_PE_v2.itp")
pg_itp        = os.path.join(lipid_itp_dir, "martini_v3.0.0_phospholipids_PG_v2.itp")
solv_itp      = os.path.join(lipid_itp_dir, "martini_v3.0.0_solvents_v1.itp")
ions_itp      = os.path.join(lipid_itp_dir, "martini_v3.0.0_ions_v1.itp")

membrane_spec = (
    "lipid:POPE:75:params:default "
    "lipid:POPG:20:params:default"
)

run_ids = [f"run_{i:02d}" for i in range(1, 4)] + ["test"]

@contextmanager
def pushd(new_dir):
    prev = os.getcwd()
    os.chdir(new_dir)
    try:
        yield
    finally:
        os.chdir(prev)

def write_ndx(filename, groups):
    """Write a GROMACS .ndx from dict[str, AtomGroup]."""
    def _chunk15(one_based_indices):
        for i in range(0, len(one_based_indices), 15):
            yield one_based_indices[i:i+15]
    with open(filename, "w") as fh:
        for name, ag in groups.items():
            fh.write(f"[ {name} ]\n")
            if len(ag) > 0:
                idx = ag.indices + 1  # to 1-based
                for block in _chunk15(idx.tolist()):
                    fh.write(" ".join(str(i) for i in block) + "\n")
            fh.write("\n")

protein_pdb_abs = os.path.abspath(protein_pdb)

for i, run_id in enumerate(run_ids, start=1):
    run_dir = os.path.join(base_dir, run_id)
    os.makedirs(run_dir, exist_ok=True)

    # Copy toppar and molecule_0.itp into the run dir (keep it simple; ignore VCS/cache dirs)
    toppar_dest = os.path.join(run_dir, "toppar")
    if os.path.exists(toppar_dest):
        shutil.rmtree(toppar_dest)
    shutil.copytree(toppar_src, toppar_dest, ignore=shutil.ignore_patterns(".git", ".github", "__pycache__"))
    shutil.copy(protein_itp, os.path.join(run_dir, os.path.basename(protein_itp)))

    print(f"\n--- Building {base_name} in {run_dir} ---\n")

    with pushd(run_dir):
        # Build the system
        COBY.COBY(
            randseed=i,
            box=[12, 12, 14],
            box_type="rectangular",
            membrane=membrane_spec,
            protein=f"file:{protein_pdb_abs} moleculetypes:{molecule_type} cz:-1.0",
            solvation="solv:W pos:NA neg:CL salt_molarity:0.15",
            itp_input=[
                f"include:{m3_core_itp}",
                f"include:{ffbonded_itp}",
                f"include:{ions_itp}",
                f"include:{solv_itp}",
                f"include:{pe_itp}",
                f"include:{pg_itp}",
                f"include:{os.path.basename(protein_itp)}",
            ],
            out_sys="output_initial_state",
            out_top="topol.top",
            out_log="log_initial_state.log",
            verbose=1,
            sn=base_name,
        )

        # Create index.ndx using MDAnalysis
        gro = "output_initial_state.gro"
        u = mda.Universe(gro)

        # Define selections
        sel_protein   = u.select_atoms("same resname as name BB")
        sel_membrane  = u.select_atoms("resname POPE POPG")
        sel_solvent   = u.select_atoms("resname W NA CL")
        # Martini protein backbone beads are typically named 'BB', sometimes BB1/BB2/BBP
        sel_prot_bb   = u.select_atoms("(name BB)")
        sel_prot_mem  = sel_protein | sel_membrane

        groups = {
            "membrane": sel_membrane,
            "protein": sel_protein,
            "protein_membrane": sel_prot_mem,
            "solvent": sel_solvent,
            "protein_bb": sel_prot_bb,
        }

        write_ndx("index.ndx", groups)

    print(f"--- Finished {base_name} in {run_dir} (index.ndx written) ---\n")



--- Building initial_state in simulations/run_01 ---

Setting random seed to: 1
-------------------------- PREPROCESSING DEFINITIONS ---------------------------
---------------------- DEFINITIONS PREPROCESSING COMPLETE ----------------------
                            (Time spent: 0.5493 [s])                            

---------------------------- PREPROCESSING COMMANDS ----------------------------
------------------------ COMMAND PREPROCESSING COMPLETE ------------------------
                            (Time spent: 0.0065 [s])                            

------------------------------ PROTEIN PLACEMENT -------------------------------
-------------------------- PROTEIN PLACEMENT COMPLETE --------------------------
                            (Time spent: 0.005 [s])                              

----------------------- CREATING LEAFLET BOUNDARY BOXES ------------------------
------------------------ LEAFLET BOUNDARY BOXES CREATED ------------------------
                        